<a href="https://colab.research.google.com/github/cdiegor/MineracaoDeDados/blob/main/Spam_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Filtro de spam usando um classificador Naive Bayes

[original](https://www.kaggle.com/code/jeffysonar/spam-filter-using-naive-bayes-classifier)

**Importar bibliotecas**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

**Leitura do arquivo CSV**

In [ ]:
df = pd.read_csv('spam.csv', encoding='latin-1')[['v1', 'v2']]
df.columns = ['label', 'message']
df.head()

**Mostrar o conjunto de dados e contagem de ham/spam**

In [ ]:
df.groupby('label').describe()

In [ ]:
sns.countplot(data=df, x='label')

**Passos para realizar o nosso filtro de spam:**

1. Limpar e normalizar o texto
2. Converter o texto em vetores de palavras (utilizando o modelo *bag of words*) para que os modelos de aprendizado consigam entender.
3. Treinar e testar o classificador.

**Limpando e normalizando o texto**
1. Remover pontuação
2. Remover *stopwords*
3. Aplicar [stemming](https://en.wikipedia.org/wiki/Stemming) (extrair a raiz das palavras).

** Método process recebe um texto e aplica os passos 1, 2 e 3, retornando uma string que consiste na lista de palavras restantes.**

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import string
from nltk.corpus import stopwords
from nltk import PorterStemmer as Stemmer
def process(text):
    # Caixa baixa
    text = text.lower()
    # Remover a pontuação
    text = ''.join([t for t in text if t not in string.punctuation])
    # Remover as stopwords
    text = [t for t in text.split() if t not in stopwords.words('english')]
    # Raiz das palavras
    st = Stemmer()
    text = [st.stem(t) for t in text]
    # Retornar o texto modificado
    return text

In [ ]:
# Testando
process('It\'s holiday and we are playing cricket. Jeff is playing very well!!!')

In [ ]:
# Testando no conjunto de dados
df['message'][:20].apply(process)

**Converta cada mensagem em vetores que os modelos de aprendizado de máquina podem entender.Faremos isso usando o modelo bag-of-words**
<br>Usaremos o TfidfVectorizer. Ele converterá a coleção de documentos de texto (corpus SMS) em uma matriz 2D.
<br>Uma dimensão representa documentos e outra dimensão representa cada palavra única no corpus SMS.

<br>Se o **n-ésimo termo t ocorreu p vezes no m-ésimo documento**, o valor (m, n) nesta matriz será TF-IDF(t), <br><center>onde [TF-IDF(t)](https://en.wikipedia.org/wiki/Tf–idf) = Frequência do termo (TF) * Frequência inversa do documento (IDF)</center>
<br>A frequência do termo (TF) é uma medida de quão frequente um termo ocorre em um documento.<br>
<br><center>TF(t)= Número de vezes que o termo t aparece no documento (p) / Número total de termos naquele documento</center>
<br>A frequência inversa do documento (IDF) é uma medida de quão importante é o termo. Para TF, todos os termos são tratados igualmente. Mas, em IDF, para palavras que ocorrem frequentemente como 'é' 'o' 'de' recebem menos peso. Enquanto termos que ocorrem raramente e que podem facilmente ajudar a identificar a classe de recursos de entrada serão ponderados alto.<br>
<br><center>Frequência de Documentos Inversa, IDF(t)= log<sub><i>e</i></sub>(Número total de documentos / Número de documentos com o termo t nele)</center>
<br>No final, teremos para cada mensagem, vetores normalizados para comprimento unitário igual ao tamanho do vocábulo (número de termos exclusivos de todo o corpus SMS)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

**Tranformando o corpus de mensagens**

In [ ]:
tfidfv = TfidfVectorizer(analyzer=process)
data = tfidfv.fit_transform(df['message'])

**Vamos ver como isto transforma uma mensagem**

In [ ]:
mess = df.iloc[2]['message']
print(mess)

In [ ]:
print(tfidfv.transform([mess]))

**Uma visão melhor**

In [ ]:
j = tfidfv.transform([mess]).toarray()[0]
print('index\tidf\ttfidf\tterm')
for i in range(len(j)):
    if j[i] != 0:
        print(i, format(tfidfv.idf_[i], '.4f'), format(j[i], '.4f'), tfidfv.get_feature_names_out()[i],sep='\t')

**Tendo mensagens em forma de vetores, estamos prontos para treinar nosso classificador. Usaremos o Naive Bayes, que é um classificador bem conhecido ao trabalhar com dados de texto. Antes disso, usaremos o recurso de pipeline do sklearn para criar um pipeline do TfidfVectorizer seguido pelo Multinomial Naive Bayes.**

A entrada será uma mensagem passada para o primeiro estágio TfidfVectorizer, que a transformará e a passará para o Naive Bayes Classifier para obter o rótulo de saída

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
spam_filter = Pipeline([
    ('vectorizer', TfidfVectorizer(analyzer=process)), # mensagens convertidas com scores tf-idf
    ('classifier', MultinomialNB())                    # treinamento com os dados do tf-idf
])

**Fazer a divisão entre treinamento e teste**

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.20, random_state = 21)

**Treinar o spam_filter**

In [ ]:
spam_filter.fit(x_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(analyzer=<function process at 0x782f03f040d0>)),
                ('classifier', MultinomialNB())])

**Predição para casos de teste**

In [ ]:
predictions = spam_filter.predict(x_test)

In [ ]:
count = 0
for i in range(len(y_test)):
    if y_test.iloc[i] != predictions[i]:
        count += 1
print('Número total de testes', len(y_test))
print('Número de erros', count)

Número total de testes 1115
Número de erros 39


**Checar quais foram os erros nas predições**

In [ ]:
x_test[y_test != predictions]

,message
419,Send a logo 2 ur lover - 2 names joined by a h...
3139,sexy sexy cum and text me im wet and warm and ...
3790,"Twinks, bears, scallies, skins and jocks are c..."
2877,Hey Boys. Want hot XXX pics sent direct 2 ur p...
2377,YES! The only place in town to meet exciting a...
1499,"SMS. ac JSco: Energy is high, but u may not kn..."
3417,LIFE has never been this much fun and great un...
3358,Sorry I missed your call let's talk when you h...
2412,I don't know u and u don't know me. Send CHAT ...
3862,Oh my god! I've found your number again! I'm s...


**Utilizar a função report para melhores detalhes**

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predictions, y_test))

              precision    recall  f1-score   support

         ham       1.00      0.96      0.98      1014
        spam       0.72      1.00      0.84       101

    accuracy                           0.97      1115
   macro avg       0.86      0.98      0.91      1115
weighted avg       0.97      0.97      0.97      1115



Olhando para a coluna de precisão (para ham, é 1,00), podemos dizer que todo o número de previsões erradas veio de spam previsto como ham. Está tudo bem e o custo de prever spam como ham é significativamente menor comparado com o de prever ham como spam.

**Função para dizer se uma mensagem é ham ou spam**

In [ ]:
def detect_spam(s):
    return spam_filter.predict([s])[0]
detect_spam('Your cash-balance is currently 500 pounds - to maximize your cash-in now, send COLLECT to 83600.')

'spam'